<a href="https://colab.research.google.com/github/DeviManognya/Assignment1/blob/main/Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision scikit-learn matplotlib pandas numpy stanza

In [ ]:
import zipfile
import os
import stanza

In [ ]:
stanza.download('en')

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: en (English) ...


INFO:stanza:Downloaded file to /root/stanza_resources/en/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources


In [ ]:
def extract_zip(zip_path, extract_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

In [ ]:
!wget http://mlg.ucd.ie/files/datasets/bbc-fulltext.zip

--2024-03-06 06:28:37--  http://mlg.ucd.ie/files/datasets/bbc-fulltext.zip
Resolving mlg.ucd.ie (mlg.ucd.ie)... 137.43.93.132
Connecting to mlg.ucd.ie (mlg.ucd.ie)|137.43.93.132|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2874079 (2.7M) [application/zip]
Saving to: ‘bbc-fulltext.zip’

bbc-fulltext.zip    100%[===================>]   2.74M  4.51MB/s    in 0.6s    

2024-03-06 06:28:38 (4.51 MB/s) - ‘bbc-fulltext.zip’ saved [2874079/2874079]



In [ ]:
extract_zip("bbc-fulltext.zip", "./")

In [ ]:
os.remove("bbc-fulltext.zip")

In [ ]:
def create_dataset():
    X = []
    y = []
    categories = ['business', 'entertainment', 'politics', 'sport', 'tech']
    for category in categories:
        folder_path = os.path.join('bbc', category)
        file_names = os.listdir(folder_path)
        for file_name in file_names:
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, 'r', encoding='utf-8') as file:
                text = file.read()
                sentences = text.split('.')
                X.extend(sentences)
                y.extend([category] * len(sentences))
    return X, y


In [ ]:
X, y = create_dataset()

In [ ]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

le = LabelEncoder()
y = to_categorical(le.fit_transform(y))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_dev, y_train, y_dev = train_test_split(X, y, test_size=0.2, random_state=42)
X_dev, X_test, y_dev, y_test = train_test_split(X_dev, y_dev, test_size=0.5, random_state=42)

In [ ]:
print("Training set lengths - X_train: {}, y_train: {}".format(len(X_train), len(y_train)))
print("Development set lengths - X_dev: {}, y_dev: {}".format(len(X_dev), len(y_dev)))
print("Test set lengths - X_test: {}, y_test: {}".format(len(X_test), len(y_test)))

Training set lengths - X_train: 36753, y_train: 36753
Development set lengths - X_dev: 4594, y_dev: 4594
Test set lengths - X_test: 4595, y_test: 4595


In [ ]:
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
X_train = embed(X_train)
X_dev = embed(X_dev)
X_test = embed(X_test)

In [ ]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
model = Sequential()
model.add(Dense(units=32, activation="relu"))
model.add(Dense(units=64, activation="relu"))
model.add(Dense(units=5, activation="softmax"))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(x=X_train, y=y_train,
          epochs=20,
          validation_data=(X_dev, y_dev),
          batch_size=10,
          verbose=1)

Epoch 1/20
3676/3676 [==============================] - 9s 2ms/step - loss: 0.6220 - accuracy: 0.7703 - val_loss: 0.5383 - val_accuracy: 0.7989
Epoch 2/20
3676/3676 [==============================] - 9s 2ms/step - loss: 0.5416 - accuracy: 0.7945 - val_loss: 0.5311 - val_accuracy: 0.8015
Epoch 3/20
3676/3676 [==============================] - 10s 3ms/step - loss: 0.5151 - accuracy: 0.8033 - val_loss: 0.5246 - val_accuracy: 0.8043
Epoch 4/20
3676/3676 [==============================] - 9s 3ms/step - loss: 0.4911 - accuracy: 0.8130 - val_loss: 0.5170 - val_accuracy: 0.8037
Epoch 5/20
3676/3676 [==============================] - 8s 2ms/step - loss: 0.4697 - accuracy: 0.8231 - val_loss: 0.5233 - val_accuracy: 0.8056
Epoch 6/20
3676/3676 [==============================] - 10s 3ms/step - loss: 0.4500 - accuracy: 0.8286 - val_loss: 0.5244 - val_accuracy: 0.8091
Epoch 7/20
3676/3676 [==============================] - 10s 3ms/step - loss: 0.4323 - accuracy: 0.8377 - val_loss: 0.5241 - val_accura

In [ ]:
import numpy as np
from sklearn.metrics import classification_report

predictions = np.argmax(model.predict(X_test), axis=-1)
y_test = le.inverse_transform([np.argmax(y) for y in y_test])  # reconstruct original string labels
predictions = le.inverse_transform(predictions)
report = classification_report(y_test, predictions)

144/144 [==============================] - 1s 3ms/step


In [ ]:
report = classification_report(y_test, predictions, output_dict=True, zero_division='warn')

In [ ]:
print("\t\tprecision\trecall\tf1-score")
for category, metrics in report.items():
    if category in ['accuracy', 'macro avg', 'weighted avg']:
        print(category, end="\t")
        print(round(metrics['precision'], 2) if isinstance(metrics, dict) else round(metrics, 2), end="\t\t")
        print(round(metrics['recall'], 2) if isinstance(metrics, dict) else "", end="\t\t")
        print(round(metrics['f1-score'], 2) if isinstance(metrics, dict) else "")
    else:
        print("{}\t{}".format(category, "\t".join([str(round(metrics[metric], 2)) for metric in ['precision', 'recall', 'f1-score']])))

		precision	recall	f1-score
business	0.71	0.81	0.76
entertainment	0.77	0.72	0.74
politics	0.78	0.79	0.78
sport	0.88	0.83	0.85
tech	0.84	0.78	0.81
accuracy	0.79				
macro avg	0.79		0.79		0.79
weighted avg	0.79		0.79		0.79
